In [1]:
import sys; sys.path.insert(0, '../../')
from definitions import *

In [2]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from src.data.dataset import TimeSeriesDataset, ListDataset
from src.data.functions import torch_ffill
from src.features.derived_features import shock_index, partial_sofa
from src.model.model_selection import stratified_kfold_cv
from src.model.nets import RNN
from src.model.optimizer import optimize_utility_threshold, compute_utility_from_indexes

# RNN Prediction Example
This notebook will explain how to setup the data to use an RNN for prediction. It follows closely the example in `src/model/examples/train_rnn.py` but includes further explanation regarding each step. 

This is designed to show how one can integrate a sequential deep-learning model into the current framework. The general procedure will involve the same basic steps, but different features will be derived and chosen by the user, and the model will most likely be replaced with something else along with an adapted training procedure. Evaluation against the utility score is tricky (it is overviewed in part 10.) and so I suggest that if you wish to use this as an evaluation method you copy and paste the evaluation part and ensure your inputs are of the required form. 

Set a GPU device to train with if one is available. 

In [4]:
# GPU
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Now load the raw data from save. This includes the TimeSeriesDataset format of the training data, and the utility score labels. 

### 1. Get Data

In [5]:
# Load the full dataset
dataset = TimeSeriesDataset().load(DATA_DIR + '/raw/data.tsd')
labels = torch.Tensor(load_pickle(DATA_DIR + '/processed/labels/utility_scores.pickle'))

Forward fill the data.

In [ ]:
dataset.data = torch_ffill(dataset.data)

### 2. Expert Features
Compute some 'expert knowledge' features.

In [ ]:
dataset['PartialSOFA'] = partial_sofa(dataset)

### 3. Time-Series Features
Get any additional features, such as signatures, mins, maxs, etc. over some rolling window that you want to put into the RNN.

In [ ]:
# dataset['MaxShockIndex'] = RollingStatistic(statistic='max', window_length=5).transform(dataset['SBP'])
# dataset['MinHR'] = RollingStatistic(statistic='min', window_length=8).transform(dataset['HR'])

We comment these out as we will not actually use any in this example. 

### 4. Choose a Subset of the Features

Most of the variables change *very* infrequently and the feature set is very large. This is not great for an RNN model, so we choose to subset only those features that are frequently changing. 

For this example we look at the vitals signs: ['DBP', 'SBP', 'Temp', 'HR', 'MAP', 'Resp'], the SOFA score ['PartialSofa'], and time ['ICULOS']

In [ ]:
dataset = dataset.subset(['DBP', 'SBP', 'Temp', 'HR', 'MAP', 'PartialSOFA', 'ICULOS'])

### 5. Fill Missing Values
RNNs cannot handle missing values, and we typically have a lot of nans (though this is not so much the case with the frequently changing features). Anyway, here we simply fill nans with the zero value.

In [ ]:
dataset.data[torch.isnan(dataset.data)] = 0

### 6. Setup DataLoaders
Now we will setup the methods to feed the data into the RNN model. To start with, convert the data from the filled tensor format onto a list of variable length tensors. `data_list[0]` will correspond to a tensor of values for a given patient. 

In [ ]:
data_list = dataset.to_list()

Now we need to setup the cross-validation procedure. This is a little tricky and will be improved. The idea is that we want the same number of 'eventual sepsis' cases in each fold, and no patient being in more than one fold. The code below is simply ensuring this. 

In [12]:
# Get the id-indexed CV fold. We need both patient indexes and time index.
cv, id_cv = stratified_kfold_cv(dataset, labels, n_splits=5, return_as_list=True, seed=1)
train_idxs, test_idxs = cv[0]
train_id_idxs, test_id_idxs = id_cv[0]

# Make train and test data
train_data = [data_list[i].to(device) for i in train_id_idxs]
train_labels = [labels[i].to(device) for i in train_idxs]
test_data = [data_list[i].to(device) for i in test_id_idxs]
test_labels = [labels[i].to(device) for i in test_idxs]

Now we put this into a PyTorch dataloading format. We make a custom `ListDataset` class such that indexing returns one of the entries from the `data_list` along with the corresponding labels for each time-point of that patient.  

In [13]:
# Datasets
train_ds = ListDataset(train_data, train_labels)
test_ds = ListDataset(test_data, test_labels)

# Dataloaders. We use a batch size of 1 as we have lists not tensors.
train_dl = DataLoader(train_ds, batch_size=1)
test_dl = DataLoader(test_ds, batch_size=1)

### 7. Setup Model
Now the data is in a standard format for training, we setup the RNN model. 

In [15]:
# Model setup
model = RNN(in_channels=dataset.size(2), hidden_channels=10, out_channels=1)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()
model.to(device)

RNN(
  (rnn): RNN(7, 10, batch_first=True)
  (linear): Linear(in_features=10, out_features=1, bias=True)
)

### 8. Train Model
Now run a standard training loop monitoring losses. 

In [ ]:
# Training loop
n_epochs = 1
print_freq = 1
model.train()
for epoch in range(n_epochs):
    train_losses = []
    for i, batch in tqdm(enumerate(train_dl)):
        optimizer.zero_grad()
        inputs, true = batch
        outputs = model(inputs)
        loss = loss_fn(outputs.view(-1), true.view(-1))
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

    if epoch % print_freq == 0:
        train_loss = np.mean(train_losses)
        print("Epoch: {:.3f}  Average training loss: {:.3f}".format(epoch, train_loss))

130it [00:00, 198.77it/s]

### 9. Generate Predictions
Now simply loop over the training and testing sets to generate the predictions for each, and concatenate everything into a single tensor.

In [ ]:
# Evaluate on test
model.eval()
train_preds, test_preds = [], []
with torch.no_grad():
    # Predict train
    for batch in train_data:
        train_preds.append(model(batch.unsqueeze(0)).view(-1))

    # Predict test
    for batch in test_data:
        test_preds.append(model(batch.unsqueeze(0)).view(-1))

# Concat
train_preds = torch.cat(train_preds).view(-1).detach()
test_preds = torch.cat(test_preds).view(-1).detach()
train_labels = torch.cat(train_labels).view(-1).detach()
test_labels = torch.cat(test_labels).view(-1).detach()

### 10. Evaluate the Performance of the Models 
Finally evaluate model performance. The easiest thing to do is evaluate the losses, but this doesn't tell you much. To evaluate properly we need to find what our score on the challenge utility function is. We give a description of what you need to do and how that is achieved below. 

**Utility function evaluation**
We have optimised our model to estimate the utility score for each timepoint for each patient. However, predictions must be given as a 0-1 binary classification as to whether a person has sepsis. Thus we need to define a threshold for which if our prediction exceeds it, we will define that patient as having sepsis. We find this threshold via a gradient free optimization procedure on the training set. The outline is as follows: 

1. Start with an initial guess (threshold = 0) and evaluate the training utility with that threshold.
2. Search around this guess using some sensible optimisation procedure, each time evaluating the utility given that new tested threshold. 
3. Continue until convergence. 

This is what is going on internally in these functions below. One thing that should be noted is that evaluating the utility from scratch for each patient takes an extremely long time, and given that we normally have around 200 iterations until convergence, it is on the order of hours before this optimization scheme converges. What we do instead is compute one time the utility one would achieve for predicting 0 at each time-point in the dataset, and predicting 1 at each point in the dataset. This is stored on disk in `data/processed/labels/full_scores.pickle`. Now when we make new predictions, provided we save the indexes those predictions correspond to in the full dataset, we can extract the score achieved (either by pulling from the 0 or 1 column of the saved scores, depending on what our prediction was at that point) from the pre-computed saved scores. Then simply sum up and normalise. 

This is just meant to explain why this evaluation is a little tricky, and the evaluation below is a little strange, and requires the cross validation indexes to be specified. 

In [ ]:
# Compute losses
train_loss = loss_fn(train_labels, train_preds)
test_loss = loss_fn(test_labels, test_preds)
print('Train loss: {:.3f}'.format(train_loss))
print('Test loss: {:.3f}'.format(test_loss))

# Compute the score on the utility function
train_idxs, test_idxs = torch.cat(train_idxs), torch.cat(test_idxs)
tfm_np = lambda x: x.cpu().numpy()
train_preds, test_preds = tfm_np(train_preds), tfm_np(test_preds)
thresh = optimize_utility_threshold(train_preds, idxs=train_idxs)
train_utility = compute_utility_from_indexes(train_preds, thresh, idxs=train_idxs)
test_utility = compute_utility_from_indexes(test_preds, thresh, idxs=test_idxs)
print('Train utility score: {:.3f}'.format(train_utility))
print('Test utility score: {:.3f}'.format(test_utility))